In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [2]:
csv_columns_names = ['sepallength','sepalwidth','petallength','petalwidth','species']
species = ["setosa",'versicolor','virginica']

train_path = tf.keras.utils.get_file(
    "iris_training.csv","http://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")

test_path = tf.keras.utils.get_file(
    "iris_test.csv","http://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train =pd.read_csv(train_path, names = csv_columns_names, header=0)
test = pd.read_csv(test_path, names = csv_columns_names, header = 0)

In [3]:
train.head()

,sepallength,sepalwidth,petallength,petalwidth,species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [4]:
test.head()

,sepallength,sepalwidth,petallength,petalwidth,species
0,5.9,3.0,4.2,1.5,1
1,6.9,3.1,5.4,2.1,2
2,5.1,3.3,1.7,0.5,0
3,6.0,3.4,4.5,1.6,1
4,5.5,2.5,4.0,1.3,1


In [5]:
train_y = train.pop("species")
test_y = test.pop("species")

In [6]:
def input_fn(features, labels, training=True, batch_size = 256):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    
    if training == True:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)    

In [7]:
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='sepallength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='sepalwidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='petallength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='petalwidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [8]:
classifier = tf.estimator.DNNClassifier(feature_columns= my_feature_columns,
                                       hidden_units=[30,10],
                                       n_classes = 3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpm6a_pzwh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
classifier.train(input_fn = lambda: input_fn(train, train_y, training=True),steps=5000)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO

In [10]:
eval_result = classifier.evaluate(input_fn = lambda: input_fn(test, test_y, training=False))
print('\nTest set Accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-05T17:44:59Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpm6a_pzwh/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.20742s
INFO:tensorflow:Finished evaluation at 2020-07-05-17:44:59
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.7, average_loss 

In [55]:
def input_fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['sepallength','sepalwidth','petallength','petalwidth']

print("Please enter the values for the following features")
predict = {}
for feature in features:
    valid = True
    while valid==True:
        val = input(feature +": ")
        if not val.isdigit(): valid =False
    predict[feature] =[float(val)]
    
print(predict)    
    

predictions = classifier.predict(input_fn=lambda:input_fn(predict))
for pre_dict in predictions:
    print(pre_dict)
    class_id = pre_dict['class_ids'][0]
    probability = pre_dict["probabilities"][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
         species[class_id],100* probability))

Please enter the values for the following features
sepallength: 4.5
sepalwidth: 5.5
petallength: 6.
petalwidth: 3.5
{'sepallength': [4.5], 'sepalwidth': [5.5], 'petallength': [6.0], 'petalwidth': [3.5]}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpm6a_pzwh/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-3.0756617 , -1.9159694 , -0.72095263], dtype=float32), 'probabilities': array([0.06791625, 0.21658167, 0.7155021 ], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "virginica" (71.6%)


In [54]:
af = test.to_dict("records")
af[0]
af[0].values()

dict_values([5.9, 3.0, 4.2, 1.5])

In [56]:
test_predictions = classifier.predict(input_fn=lambda:input_fn(af))
# for test_pred in test_predictions:
#     print(test_pred)
# print(next(test_predictions))

